### Źródła:
<a href="https://www.kaggle.com/datasets/siddharthkumar25/malicious-and-benign-urls" target="_blank"> Malicious And Benign URLs  </a> <br/>
<a href="https://www.kaggle.com/datasets/sid321axn/malicious-urls-dataset" target="_blank"> Malicious URLs dataset  </a>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd